## Genetic Algorithm

The algorithm uses analogs of a genetic representation (bitstrings), fitness (function evaluations), genetic recombination (crossover of bitstrings), and mutation (flipping bits).

In [1]:
import numpy as np
import pandas as pd
from random import randint

#### Helper Functions

In [2]:
def binary_array_to_int(bin_array):
  length = len(bin_array)
  result = 0
  for i in range(length):
    result += 2**(length - i -1)*bin_array[i]
  return result

def int_to_binarry_array(integer, array_length):
  result = []
  for i in range(array_length):
    result.append(integer %2)
    integer = int(integer/2)
  result.reverse()
  return result


def sort_and_nomalize_individuals_scores(individuals_scores, log_mode):
  # sort individuals based on their score and sort them in the descending order
  sorted_individuals_scores = sorted(individuals_scores, key=lambda x: x[0])[::-1]
  raw_scores = [i[0] for i in sorted_individuals_scores]

  moved_scores = [i + 2*abs(min(raw_scores)) for i in raw_scores]
  normalized_scores = [round(i /sum(moved_scores), 2) for i in moved_scores]
  
  #print('raw sorted scores: ', raw_scores)
  #print('normalized sorted scores: ', normalized_scores)
  sort_and_nomalize_individuals_scores = []
  
  for i in range(len(normalized_scores)):
    sort_and_nomalize_individuals_scores.append((normalized_scores[i] , sorted_individuals_scores[i][1]))
  log(f'individuals and their normalized fitness scores, sorted by scores: {sort_and_nomalize_individuals_scores}', log_mode)
  return sort_and_nomalize_individuals_scores

def log(string, log_mode):
  if log_mode:
    print(string)


#### Getting Inputs

In [3]:
indiv_length = 5 # because it should be between 0 and 30, which is 2^5
n_population = int(input('Enter the population number: '))
mutat_probability = float(input('Enter mutation probability: '))
comb_probability = float(input('Enter combination probability: '))
comb_type = int(input('Enter combination type, 1 for one-point and 2 for uniform: '))
max_loop_iter = int(input('Enter maximum number of loop iteration: '))
log_mode = True

ValueError: invalid literal for int() with base 10: ''

#### Initialize the Population

In [ ]:
def initialize_populatoin(indiv_length, n_population, log_mode):
  population = [np.random.randint(0, 2, indiv_length) for _ in range(n_population)]
  log(f'initial population: {population}', log_mode)
  return population

#### Fitness Function

In [ ]:
def fitness(individual):
  individual_integer_form = binary_array_to_int(individual)
  # -x^2 + 6x
  score = - individual_integer_form**2 + 6*individual_integer_form
  return score

#### Parent Selection

In [ ]:
def roulette_wheel_selection(population, log_mode):
  cumulative_propapilities = []
  for i in range(len(population)):
    #print('individual: ', population[i])
    if i ==0:
      cumulative_propapilities.append(population[i][0])
    else:
      cumulative_propapilities.append(round(population[i][0] + cumulative_propapilities[i-1], 2))
  cumulative_propapilities[len(cumulative_propapilities)-1] = 1.0
  log(f'roulette wheel(cumulative_propapilities): {cumulative_propapilities}', log_mode)

  # roulette wheel 
  parents = []
  for _ in range(len(population)):
    random_num = np.random.random()
    for i in range(len(cumulative_propapilities)):
      if random_num < cumulative_propapilities[i]:
        parents.append(population[i][1])
        break
  log(f'parents of new population: {parents}', log_mode)
  return parents

#### Crossover

In [ ]:
def crossover(parents, indiv_length, comb_probability, comb_type, log_mode):
  log('combination...', log_mode)
  crossover_result = []

  #select pairs
  for i in range(int(len(parents)/2)):
    random_comb = np.random.random()
    if(random_comb < comb_probability):
      # add tolist() because chromosomes are of np.array form 
      parent1 = parents[i].tolist()
      parent2 = parents[len(parents)-1 -i].tolist()
      # 1-point crossover
      if comb_type == 1:  
        random_index = np.random.randint(1, indiv_length-1)
        child1 = np.array(parent1[:random_index]+parent2[random_index:])
        child2 = np.array(parent2[:random_index]+parent1[random_index:])
        crossover_result.append(child1)
        crossover_result.append(child2)
        log(f'random index is: {random_index}', log_mode)
        log(f'parent1: {parent1[:random_index]}{parent1[random_index:]}', log_mode)
        log(f'parent2: {parent2[:random_index]}{parent2[random_index:]}', log_mode)
        log(f'child1: {parent1[:random_index]+parent2[random_index:]}', log_mode)
        log(f'child2: {parent2[:random_index]+parent1[random_index:]}', log_mode)

      # uniform crossover
      elif comb_type==2: 
        parent1 = parents[i].tolist()
        parent2 = parents[len(parents)-1 -i].tolist()
        child1 = []
        child2 = []
        for j in range(indiv_length):
          random_result = np.random.random()
          if random_result <.5:
            child1.append(parent1[j])
            child2.append(parent2[j])
          else:
            child1.append(parent2[j])
            child2.append(parent1[j])
          
        crossover_result.append(np.array(child1))
        crossover_result.append(np.array(child2))
        log(f'parent1: {parent1}', log_mode)
        log(f'parent2: {parent2}', log_mode)
        log(f'child1: {child1}', log_mode)
        log(f'child2: {child2}', log_mode)


    else:
      crossover_result.append(parents[i])
      crossover_result.append(parents[len(parents)-1-i])
      log(f'not perfom crossover for the {i}th pair or chromosomes because the outcome of random is {round(random_comb)}, where as combination probability is: {comb_probability}', log_mode)
      log(f'child1: {parents[i].tolist()} (sam as parent)', log_mode)
      log(f'child1: {parents[len(parents)-1 -i].tolist()} (same as parent)', log_mode)
  return crossover_result
crossover([np.array([1, 0, 0, 0, 1]),np.array([0, 1, 1, 0, 1]), np.array([0, 1, 0, 1, 1]),np.array([1, 1, 1, 1, 1])], 5, .7, 2 , log_mode)

NameError: name 'log_mode' is not defined

#### Mutation

In [ ]:
def mutation(parents, indiv_length, mutat_probability, log_mode):
  log('mutation...', log)
  mutation_result = []
  #select pairs
  for i in range(int(len(parents))):
    parent = parents[i].tolist()
    child = []
    for j in range(indiv_length):
      random_result = np.random.random()
      #print(f'random is: {round(random_result, 2)} and mutation probability is: {mutat_probability}')
      if random_result < mutat_probability:
        # mutation happens for gene #j
        child.append(1-parent[j])
      else:
        child.append(parent[j])
    mutation_result.append(np.array(child))
    log(f'parent: {parent}', log_mode)
    log(f'child: {child}', log_mode)

  return mutation_result
mutation([np.array([1, 0, 0, 0, 1]),np.array([0, 1, 1, 0, 1]), np.array([0, 1, 0, 1, 1]),np.array([1, 1, 1, 1, 1])], 5, .6 , True)

NameError: name 'np' is not defined

#### Genetic Algorithm

In [ ]:
def GA(indiv_length, n_population, mutat_probability, comb_probability, comb_type, max_loop_iter, log_mode):
  population = initialize_populatoin(indiv_length, n_population, log_mode)
  best_score = -100000
  fittest_individual = None
  for _ in range(max_loop_iter):
    log(f'population of gen#{_}: {population}', log_mode)
    individuals_scores = []
    for individual in population:
      score = fitness(individual)
      if score > best_score:
        best_score, fittest_individual= score, individual
      individuals_scores.append((score, individual))
    sorted_and_nomalized_individuals_scores = sort_and_nomalize_individuals_scores(individuals_scores, log_mode)
    #selection
    parents = roulette_wheel_selection(sorted_and_nomalized_individuals_scores, log_mode)
    #crossover
    crossover_result = crossover(parents, indiv_length, comb_probability, comb_type, log_mode)
    #mutation
    mutation_result = mutation(crossover_result, indiv_length, mutat_probability, log_mode)
  
    population = mutation_result
    
  
  log(f'fittest individual is: {binary_array_to_int(fittest_individual)}, binarry representation: {fittest_individual.tolist()}, wih score: {best_score}', log_mode)

GA(indiv_length, n_population, mutat_probability, comb_probability, comb_type, max_loop_iter, log_mode)

NameError: name 'indiv_length' is not defined